<a href="https://colab.research.google.com/github/timthedev07/news-articles-classification/blob/dev/news_articles_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [25]:
import os
import string
import re
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, LSTM
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
stopWords = set(stopwords.words("english"))
stopWords.remove("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load data.

In [17]:
!rm -rf ./sample_data
!rm -rf ./data
!mkdir data

from google.colab import drive
drive.mount('/content/gdrive')

!cp "/content/gdrive/My Drive/datasets/news-articles-categorization/data.json.zip" data
%cd data
!unzip -ojq data.json.zip
!mv News_Category_Dataset_v2.json data.json
!rm -rf data.json.zip
%cd ..

data = pd.read_json("data/data.json", lines = True)

In [22]:
x = data["headline"]
y = pd.get_dummies(data["category"])

trainX, testX, trainY, testY = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)

LABELS = np.sort(data["category"].unique())

Input cleaning.

In [23]:
def customStandardization(text: tf.Tensor):
    # to lower case
    text = tf.strings.lower(text)
    # expand contraction
    pairs = [
        ("won't", "will not"),
        ("can't", "can not"),
        ("n't", " not"),
        ("'re", " are"),
        ("'s", " is"),
        ("'d", " would"),
        ("'ll", " will"),
        ("'t", " not"),
        ("'ve", " have"),
        ("'m", " am"),
    ]
    for contracted, replacement in pairs:
        text = tf.strings.regex_replace(text, contracted, replacement)
    
    # clean special symbols
    text = tf.strings.regex_replace(text, r"\d+(?:\.\d*)?(?:[eE][+-]?\d+)?", " ")
    text = tf.strings.regex_replace(text, r'@([A-Za-z0-9_]+)', " ")
    text = tf.strings.regex_replace(text, r"[^A-Za-z0-9]+", " ")

    # remove stopwords
    for i in stopWords:
        text = tf.strings.regex_replace(text, f"[^A-Za-z0-9_]+{i}[^A-Za-z0-9_]+", " ")

    return text

The model.

In [28]:
SEQUENCE_LENGTH = 20
VOCAB_SIZE = 15000

vectorizeLayer = TextVectorization(
        standardize=customStandardization,
        max_tokens=VOCAB_SIZE,
        output_mode='int',
        output_sequence_length=SEQUENCE_LENGTH)

vectorizeLayer.adapt(np.concatenate([trainX, testX]))

In [31]:
model = Sequential([
    vectorizeLayer,
    Embedding(VOCAB_SIZE, 32, name="embedding"),
    LSTM(256, name="lstm_1", return_sequences = True),
    LSTM(128, name="lstm_2"),
    Dense(len(LABELS), activation = "softmax")
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, 20)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 20, 32)            480000    
                                                                 
 lstm_1 (LSTM)               (None, 20, 256)           295936    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dense_4 (Dense)             (None, 41)                5289      
                                                                 
Total params: 978,345
Trainable params: 978,345
Non-trainable params: 0
________________________________________________